<font size="6">Prétraitement des données</font>

Avant de plonger dans l'analyse et la modélisation de données, une étape cruciale dans tout projet de science des données est le prétraitement des données. Le prétraitement vise à  préparer les données brutes de manière à ce qu'elles soient prêtes à être explorées et analysées. Cette phase est essentielle pour garantir la qualité et la fiabilité de nos résultats.

Le prétraitement des données consiste en plusieurs tâches, notamment la gestion des valeurs manquantes, la suppression du bruit et la transformation des données. L'objectif est de créer un ensemble de données nettoyé et structuré, prêt à être utilisé pour construire des modèles prédictifs ou pour effectuer des analyses approfondies.

Dans cette section de notre rapport, nous allons passer en revue les différentes étapes de prétraitement des données que nous avons entreprises pour notre projet. Nous expliquerons les raisons derrière chaque étape, les techniques utilisées et les outils mis en œuvre. De plus, nous détaillerons comment ces actions contribuent à améliorer la qualité de nos données et à rendre notre travail d'analyse plus efficace.

In [36]:
import pandas as pd

# ! options !
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)


In [37]:
## Import Data ##

chemin_fichier_2018 = '../valeursfoncieres-2018.txt'
chemin_fichier_2019 = '../valeursfoncieres-2019.txt'
chemin_fichier_2020 = '../valeursfoncieres-2020.txt'
chemin_fichier_2021 = '../valeursfoncieres-2021.txt'


df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|')
df_19 = pd.read_csv(chemin_fichier_2019, delimiter='|')
df_20 = pd.read_csv(chemin_fichier_2020, delimiter='|')
df_21 = pd.read_csv(chemin_fichier_2021, delimiter='|')

C:\Users\nagrimault\AppData\Local\Temp\ipykernel_3860\445314122.py:9: DtypeWarning: Columns (14,18,23,24,26,28,29,31,32,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_18 = pd.read_csv(chemin_fichier_2018, delimiter='|')
C:\Users\nagrimault\AppData\Local\Temp\ipykernel_3860\445314122.py:10: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_19 = pd.read_csv(chemin_fichier_2019, delimiter='|')
C:\Users\nagrimault\AppData\Local\Temp\ipykernel_3860\445314122.py:11: DtypeWarning: Columns (18,23,24,26,28,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_20 = pd.read_csv(chemin_fichier_2020, delimiter='|')
C:\Users\nagrimault\AppData\Local\Temp\ipykernel_3860\445314122.py:12: DtypeWarning: Columns (18,23,24,26,28,29,30,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df_21 = pd.read_csv(chemin_fichier_202

In [38]:
# Concat all years in one dataframe (and take some parts if we want)

df_18 = df_18.sample(frac=1, random_state=42)
df_19 = df_19.sample(frac=1, random_state=42)
df_20 = df_20.sample(frac=1, random_state=42)
df_21 = df_21.sample(frac=1, random_state=42)

df = pd.concat([df_18, df_19, df_20, df_21], axis=0)


In [39]:
# Replace ' ' by '_'

df.columns = df.columns.str.replace(' ', '_')

Commençons par visualiser quelques lignes de notre Data Frame :

In [40]:
df.head()

,Identifiant_de_document,Reference_document,1_Articles_CGI,2_Articles_CGI,3_Articles_CGI,4_Articles_CGI,5_Articles_CGI,No_disposition,Date_mutation,Nature_mutation,Valeur_fonciere,No_voie,B/T/Q,Type_de_voie,Code_voie,Voie,Code_postal,Commune,Code_departement,Code_commune,Prefixe_de_section,Section,No_plan,No_Volume,1er_lot,Surface_Carrez_du_1er_lot,2eme_lot,Surface_Carrez_du_2eme_lot,3eme_lot,Surface_Carrez_du_3eme_lot,4eme_lot,Surface_Carrez_du_4eme_lot,5eme_lot,Surface_Carrez_du_5eme_lot,Nombre_de_lots,Code_type_local,Type_local,Identifiant_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Nature_culture_speciale,Surface_terrain
1883842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,25/05/2018,Vente,"7500,00",11.00,NaN,RUE,5620,DES SABLONS,58000.00,NEVERS,58,194,NaN,AS,242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,AB,NaN,479.00
2050544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,26/07/2018,Vente,"60000,00",NaN,NaN,NaN,B008,LES COUTURES,62149.00,ANNEQUIN,62,34,NaN,AC,171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,J,NaN,733.00
3256050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,22/08/2018,Vente,"45000,00",105.00,NaN,RUE,1180,DE LA REPUBLIQUE,97400.00,SAINT DENIS,974,11,NaN,AH,140,NaN,244,"19,86",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.00,Appartement,NaN,20.00,1.00,NaN,NaN,NaN
2779968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,13/03/2018,Vente,"100000,00",2.00,B,TRA,1961,PIERRE RENAUDEL,83400.00,HYERES,83,69,NaN,CN,259,NaN,18.00,"47,37",2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.00,Appartement,NaN,52.00,3.00,NaN,NaN,NaN
1957212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,12/03/2018,Vente,"102000,00",50.00,NaN,RUE,0360,JULES ROCH,59310.00,ORCHIES,59,449,NaN,D,1099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.00,Maison,NaN,54.00,2.00,S,NaN,103.00


### Traitement des valeurs manquantes

Nous pouvons déjà remarquer plusieurs valeurs manquantes dans plusieurs de nos variables. Vérifions le pourcentage de valeurs manquantes pour chacune des colonnes de notre data frame.

In [41]:
# Affichez les pourcentages de NaN pour chaque colonne

pourcentages_nan = (df.isna().sum() / len(df)) * 100
print(pourcentages_nan)

Identifiant_de_document      100.00
Reference_document           100.00
1_Articles_CGI               100.00
2_Articles_CGI               100.00
3_Articles_CGI               100.00
4_Articles_CGI               100.00
5_Articles_CGI               100.00
No_disposition                 0.00
Date_mutation                  0.00
Nature_mutation                0.00
Valeur_fonciere                1.10
No_voie                       39.39
B/T/Q                         95.57
Type_de_voie                  41.25
Code_voie                      0.85
Voie                           0.85
Code_postal                    0.85
Commune                        0.00
Code_departement               0.00
Code_commune                   0.00
Prefixe_de_section            95.32
Section                        0.00
No_plan                        0.00
No_Volume                     99.74
1er_lot                       68.84
Surface_Carrez_du_1er_lot     91.48
2eme_lot                      92.63
Surface_Carrez_du_2eme_lot  

Certaines variables (Identifiant_de_document, Reference_document et les Articles_CGI) ont 100% de valeurs manquantes. Pour le moment, nous choisissons de garder les variables suivantes :

'No_disposition', 'Date_mutation', 'Nature_mutation', 'Valeur_fonciere', 'No_voie', 'Type_de_voie', 'Code_voie', 'Voie', 'Code_postal','Commune', 'Code_departement', 'Code_commune', 'Section', 'No_plan',    'Nombre_de_lots', 'Code_type_local', 'Surface_reelle_bati','Nombre_pieces_principales', 'Nature_culture', 'Surface_terrain'    

In [42]:
# Remove empty columns and keep only 'Vente' 

columns_to_remove = ["Identifiant_de_document", "Reference_document", "1_Articles_CGI", "2_Articles_CGI", "3_Articles_CGI", "4_Articles_CGI", "5_Articles_CGI",
                     "B/T/Q", "Prefixe_de_section", "No_Volume", "Type_local", "Identifiant_local", "Nature_culture_speciale",
                     "1er_lot", "Surface_Carrez_du_1er_lot", "2eme_lot", "Surface_Carrez_du_2eme_lot", "3eme_lot", "Surface_Carrez_du_3eme_lot", "4eme_lot",
                     "Surface_Carrez_du_4eme_lot", "5eme_lot", "Surface_Carrez_du_5eme_lot"]


df = df.drop(columns_to_remove, axis=1)


In [43]:
print("Taille de notre jeu de données après suppression des variables avec trop de valeurs manquantes : ")
print(df.shape)

Taille de notre jeu de données après suppression des variables avec trop de valeurs manquantes : 
(15125102, 20)


On enlève également les ventes avec des valeurs manquantes dans la variable Valeur foncière car c'est la variable que nous voulons prédire.

In [44]:
df = df.dropna(subset=['Valeur_fonciere'])

In [45]:
print("Taille de notre jeu de données après suppression des valeurs manquantes de Valeur_fonciere : ")
print(df.shape)

Taille de notre jeu de données après suppression des valeurs manquantes de Valeur_fonciere : 
(14959015, 20)


Nous supprimons également les lignes de notre data frame avec des valeurs manquantes dans le Code Postal

In [46]:
df = df.dropna(subset=['Code_postal'])
df['Code_postal'] = df['Code_postal'].astype(int)
df['Code_postal'] = df['Code_postal'].astype(object)  # puis en objet

In [47]:
print("Taille de notre jeu de données après suppression des valeurs manquantes de Code_postal : ")
print(df.shape)

Taille de notre jeu de données après suppression des valeurs manquantes de Code_postal : 
(14830324, 20)


Pour faire une classification de le type de local nous devons également supprimer les valeurs manquantes de cette variables.

In [48]:
# Drop the Nan remainings, for the classification model

df = df.dropna(subset=["Code_type_local"])
df['Code_type_local'] = df['Code_type_local'].astype(int)

In [49]:
print("Taille de notre jeu de données après suppression des valeurs manquantes de Code_type_local : ")
print(df.shape)

Taille de notre jeu de données après suppression des valeurs manquantes de Code_type_local : 
(8518444, 20)


### Traitement des natures de mutations

Regardons maintenant la variable "Nature_mutation" qui décrit la nature des ventes que nous étudions. Voici les différentes modalités de cette variables :

In [50]:
print("Modalités de la variable Nature_mutation : ")
print(df['Nature_mutation'].unique())

Modalités de la variable Nature_mutation : 
['Vente' 'Vente terrain à bâtir' "Vente en l'état futur d'achèvement"
 'Adjudication' 'Echange' 'Expropriation']


In [51]:
print("Nombre de vente pour chaque nature de mutation : ")
print(df['Nature_mutation'].value_counts())

Nombre de vente pour chaque nature de mutation : 
Nature_mutation
Vente                                 8091604
Vente en l'état futur d'achèvement     358116
Adjudication                            34312
Echange                                 29163
Vente terrain à bâtir                    4673
Expropriation                             576
Name: count, dtype: int64


Nous décidons de garder seulement les ventes pour nettoyer notre data frame.

In [52]:
df = df[df['Nature_mutation'] == "Vente"] 
df = df.drop(columns=['Nature_mutation'])

In [53]:
print("Taille de notre jeu de données en gardant seulement les ventes : ")
print(df.shape)

Taille de notre jeu de données en gardant seulement les ventes : 
(8091604, 19)


### Duplications ventes

Après visualisation de nos données on peut se rendre compte que certaines ventes sont dupliquées et représentent plusieurs lignes. 
Nous avons donc décidé de créer une variable "Adresse" avec "No_voie", "Type_de_voie", "Code_voie", "Voie", "Code_commune" et "Code_departement". Cette variable est regroupée avec la date de la vente et nous supprimons les lignes avec la même date et même adressee.

In [54]:
# Create variable 'adresse' and remove variables use to build the adress

columns_for_adress = ["No_voie", "Type_de_voie", "Code_voie",
                      "Voie", "Code_commune", "Code_departement"]


df['Code_postal'] = df.groupby('Commune')['Code_postal'].transform(lambda x: x.fillna(method='ffill').fillna(method='bfill'))  # match les code postal vide avec les communes (bien)
    
df['No_voie'] = df['No_voie'].apply(lambda x: str(int(x)) if pd.notnull(
        x) else '').astype(str)  # change le type si non il y a une valeur sinon affiche rien

df['adresse'] = df['No_voie'] + ' ' + df['Type_de_voie'].astype(str) + ' ' + \
df['Code_voie'].astype(str) + ' ' + df['Voie'].astype(str) + ' ' + \
df['Code_postal'].astype(str) + ' ' + df['Commune']
    
    
# Après la création de l'adresse on remove les variables concatenées et les autres inutiles
df = df.drop(columns_for_adress, axis=1)

In [55]:
# Remove duplicates (date_mutation et adresse)

df = df.drop_duplicates(subset=["Date_mutation", "adresse"])

# Variable 'adresse' useless now => remove
df = df.drop("adresse", axis=1)

In [56]:
print("Taille de notre jeu de données en supprimant les duplications de vente : ")
print(df.shape)

Taille de notre jeu de données en supprimant les duplications de vente : 
(4708687, 13)


In [57]:
# Affichez le nombre d'individus par modalité de la colonne 'Code_type_local'

comptage_modalites = df['Code_type_local'].value_counts()
print("Nombre de vente par type local")
print(comptage_modalites)


Nombre de vente par type local
Code_type_local
1    2134129
2    1263375
3    1047694
4     263489
Name: count, dtype: int64


In [58]:
#print("before : ", len(df))

df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)  
    
#print("after : ", len(df), "\n")

comptage_modalites = df['Code_type_local'].value_counts()
#print(comptage_modalites)


In [59]:
# masque_nan = df.isna()

# # Utilisez sum() pour compter le nombre de valeurs manquantes (True) dans chaque colonne
# nombre_nan_par_colonne = masque_nan.sum()
# print(nombre_nan_par_colonne)

### Outliers

Nous nous occupons maintenant des outliers. Les outliers peuvent fausser les analyses statistiques, affecter la performance des modèles prédictifs et altérer la visualisation des données. En les identifiant et les gérant correctement, on améliore la qualité des données, et on réduit les risques d'erreurs.

Nous décidons donc de suprimmer les ventes avec une valeurs foncière inférieure ou supérieure aux valeurs limites que nous allons définir.

In [60]:
# Remove individus with at least one NaN and outliers

print("Nombre de ventes avant suppression des outliers : ", len(df),"\n")

# OUTLIERS 
df['Valeur_fonciere'] = df['Valeur_fonciere'].astype(str).str.replace(',', '.').astype(float)

Q1 = df['Valeur_fonciere'].quantile(0.25)
Q3 = df['Valeur_fonciere'].quantile(0.75)
    
#print(Q1)
#print(Q3)

IQR = Q3 - Q1

# Définissez les limites supérieure et inférieure pour détecter les outliers
limite_inferieure = Q1 - 0.3 * IQR
limite_superieure = Q3 + 1.7 * IQR

print("Borne Inférieure")
print(limite_inferieure,"\n")

print("Borne Supérieure")
print(limite_superieure,"\n")
    
df = df[(df['Valeur_fonciere'] >= limite_inferieure) & (df['Valeur_fonciere'] <= limite_superieure)]
    
print("Nombre de ventes après suppression des outliers : ", len(df), "\n")


Nombre de ventes avant suppression des outliers :  4708687 

Borne Inférieure
44000.0 

Borne Supérieure
554000.0 

Nombre de ventes après suppression des outliers :  4012271 



### Traitement de la date

Nous voulons créer les variables "day", "month" et "year" à partir de la variable "Date_mutation".

In [61]:
# Créer la variable "day", "month" and "year" 

# NB : Utile si on la mets en 'int' et pas en objet, car le prix de l'immobilier augmente chaque année, donc plus l'année augmente plus le prix aussi
# alors qu'en 'objet' inutile car dans ce sont les data de 2022 qui vont être testés (à revoir quand même)


df['Date_mutation'] = pd.to_datetime(df['Date_mutation'], format='%d/%m/%Y')

# Extract the year component and create a new 'year' column
df['day'] = df['Date_mutation'].dt.day
df['day'] = df['day'].astype(object)

df['month'] = df['Date_mutation'].dt.month
df['month'] = df['month'].astype(object)

df['year'] = df['Date_mutation'].dt.year
df['year'] = df['year'].astype(object)

df = df.drop("Date_mutation", axis=1)
    
print("Nouvelle structure des variables de notre data frame : ")
print(df.dtypes)


Nouvelle structure des variables de notre data frame : 
No_disposition                 int64
Valeur_fonciere              float64
Code_postal                    int64
Commune                       object
Section                       object
No_plan                        int64
Nombre_de_lots                 int64
Code_type_local                int32
Surface_reelle_bati          float64
Nombre_pieces_principales    float64
Nature_culture                object
Surface_terrain              float64
day                           object
month                         object
year                          object
dtype: object


In [62]:
# Load longitude, latitude, 'Code_postal' dataframe

geo = pd.read_csv('../data/communes-departement-region.csv')

geo.head()


,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.15,4.93,1.00,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.00,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.01,5.43,2.00,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.00,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.96,5.37,4.00,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.00,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,46.00,4.91,5.00,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.00,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.75,5.59,6.00,NaN,Ambléon,Ambléon,1,Ain,84.00,Auvergne-Rhône-Alpes


In [63]:
geo.rename(columns={'code_postal': 'Code_postal'}, inplace=True)
geo.rename(columns={'nom_commune_postal': 'Commune'}, inplace=True)
geo.rename(columns={'code_departement': 'Code_departement'}, inplace=True)

In [64]:
# voir les doublons

doublons = geo[geo.duplicated(subset=['Code_postal', 'Commune'], keep=False)]
# OK -> supprmime les doublons Code postal => Commune, car quelques fois il y a une 'petite' commune dedans la grande enfin bref
geo = geo.drop_duplicates(subset=['Code_postal', 'Commune'])


In [65]:
geo['Commune'] = geo['Commune'].str.replace('-', ' ') 
df['Commune'] = df['Commune'].str.replace('-', ' ')
geo['Commune'] = geo['Commune'].str.replace('ST', 'SAINT')  # format 'ST' et 'SAINT'
df['Commune'] = df['Commune'].str.replace('ST', 'SAINT')
geo['Commune'] = geo['Commune'].str.replace('\'', ' ')
df['Commune'] = df['Commune'].str.replace('\'', ' ')

# geo_commune = geo[['Commune', 'Code_postal', 'latitude', 'longitude']]
geo_commune = geo[['code_commune_INSEE','Commune', 'Code_postal', 'latitude', 'longitude']]

# MERGE
df = pd.merge(df, geo_commune, on=['Commune', 'Code_postal'], how='left')  # OK -> merge la commune et le code postal, car desfois la commune est dans différent département 


# Le reste des communes non trouvé c'est des erreurs dans le fichier geo, exemple => LA SALLE DES ALPES, dans les data => "LA SALLE", IMPOSSIBLE À gérer ce genre d'erreur humaine => à enlever
print(df['longitude'].isna().sum()," individus avec un nom de commune pas trouvé, on les enlève du dataset")

# Drop not found
df = df.dropna(subset=['longitude'])

print("nombre final d'individus : ",len(df))

df.head()


233979  individus avec un nom de commune pas trouvé, on les enlève du dataset
nombre final d'individus :  3778292


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,code_commune_INSEE,latitude,longitude
0,1,45000.00,97400,SAINT DENIS,AH,140,1,2,20.00,1.00,NaN,NaN,22,8,2018,97411,-20.93,55.45
1,1,100000.00,83400,HYERES,CN,259,2,2,52.00,3.00,NaN,NaN,13,3,2018,83069,43.10,6.19
2,1,102000.00,59310,ORCHIES,D,1099,0,1,54.00,2.00,S,103.00,12,3,2018,59449,50.47,3.24
3,1,297000.00,94300,VINCENNES,C,16,2,2,46.00,3.00,NaN,NaN,28,9,2018,94080,48.85,2.44
4,1,74000.00,44000,NANTES,EY,28,2,2,36.00,2.00,NaN,NaN,9,6,2018,44109,47.23,-1.55


In [66]:
# Handle 'Corse' Département

def assign_department(code_postal):
    if 20000 <= code_postal <= 20190:
        return '2A' # Pour gérer la corse sud
    elif 20200 <= code_postal <= 20270:
        return '2B' # Pour gérer la corse nord
    elif len(str(code_postal)) == 4:
        dep = '0' + str(code_postal)[:1] # Pour gérer les 10 premiers département
        return dep
    else:
        dep = str(code_postal)[:2]
        return dep 


# Appliquez la fonction à la colonne 'Code_postal' en utilisant apply()
df['Departement'] = df['Code_postal'].apply(lambda x: assign_department(x))
df['Departement'] = df['Departement'].astype(str)
df.head()

,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,code_commune_INSEE,latitude,longitude,Departement
0,1,45000.00,97400,SAINT DENIS,AH,140,1,2,20.00,1.00,NaN,NaN,22,8,2018,97411,-20.93,55.45,97
1,1,100000.00,83400,HYERES,CN,259,2,2,52.00,3.00,NaN,NaN,13,3,2018,83069,43.10,6.19,83
2,1,102000.00,59310,ORCHIES,D,1099,0,1,54.00,2.00,S,103.00,12,3,2018,59449,50.47,3.24,59
3,1,297000.00,94300,VINCENNES,C,16,2,2,46.00,3.00,NaN,NaN,28,9,2018,94080,48.85,2.44,94
4,1,74000.00,44000,NANTES,EY,28,2,2,36.00,2.00,NaN,NaN,9,6,2018,44109,47.23,-1.55,44


### Ajout d'Open Data

Nous pensons que le niveau de vie d'une commune peut être un facteur important pour prédire la valeur foncière des biens immobiliers.
C'est pourquoi nous décidons d'ajouter cette information à notre Data Frame.


In [67]:
# Rajouter open data 'niveau_vie'


niveau_vie = pd.read_excel('../data/Niveau_de_vie_2017_par_comune.xlsx')
niveau_vie.rename(columns={'Code Commune': 'code_commune_INSEE'}, inplace=True)
niveau_vie.rename(columns={'Nom Commune': 'Commune'}, inplace=True)
niveau_vie.rename(columns={'Niveau de vie Commune': 'niveau_vie_commune'}, inplace=True)
niveau_vie.rename(columns={'Niveau de vie Département': 'niveau_vie_dep'}, inplace=True)


# Ajouter un zéro devant les valeurs de la colonne "Code_postal" si leur longueur est de 4
df['code_commune_INSEE'] = df['code_commune_INSEE'].astype(str).str.zfill(5)

niveau_vie = niveau_vie[['code_commune_INSEE', 'niveau_vie_commune']]
print(niveau_vie)

# Exporter en pickle pour le déploiement
niveau_vie.to_pickle('../data/niveau_vie.pkl')

df = pd.merge(df, niveau_vie, on=['code_commune_INSEE'], how='left')




      code_commune_INSEE  niveau_vie_commune
0                  05047            10021.25
1                  26142            10215.00
2                  11317            10908.50
3                  11384            11485.17
4                  30153            11680.00
...                  ...                 ...
36567              91526                 NaN
36568              74203                 NaN
36569              78264                 NaN
36570              78606                 NaN
36571              78439                 NaN

[36572 rows x 2 columns]


Pour certaines communes, le niveau de vie n'est pas renseigné. Nous traitons donc ces valeurs manquantes en les remplaçant par la median du niveau de vie.

In [68]:
# Remplace NaN 'niveau_vie'

median_niveau_vie = df['niveau_vie_commune'].median()
df['niveau_vie_commune'].fillna(median_niveau_vie, inplace=True)


#print(df['niveau_vie_commune'].isna().sum())

In [69]:
print("Aperçu de notre jeu de données final que nous utiliserons pour la prédiction des valeurs foncières")
df.head()

Aperçu de notre jeu de données final que nous utiliserons pour la prédiction des valeurs foncières


,No_disposition,Valeur_fonciere,Code_postal,Commune,Section,No_plan,Nombre_de_lots,Code_type_local,Surface_reelle_bati,Nombre_pieces_principales,Nature_culture,Surface_terrain,day,month,year,code_commune_INSEE,latitude,longitude,Departement,niveau_vie_commune
0,1,45000.00,97400,SAINT DENIS,AH,140,1,2,20.00,1.00,NaN,NaN,22,8,2018,97411,-20.93,55.45,97,19781.43
1,1,100000.00,83400,HYERES,CN,259,2,2,52.00,3.00,NaN,NaN,13,3,2018,83069,43.10,6.19,83,19910.33
2,1,102000.00,59310,ORCHIES,D,1099,0,1,54.00,2.00,S,103.00,12,3,2018,59449,50.47,3.24,59,19646.67
3,1,297000.00,94300,VINCENNES,C,16,2,2,46.00,3.00,NaN,NaN,28,9,2018,94080,48.85,2.44,94,30584.00
4,1,74000.00,44000,NANTES,EY,28,2,2,36.00,2.00,NaN,NaN,9,6,2018,44109,47.23,-1.55,44,20725.65


In [70]:
# exporter DF
df.to_pickle('../data/clean_df.pkl')
